- https://docs.rapids.ai/api/cudf/stable/10min.html
- https://www.dataquest.io/blog/data-science-portfolio-machine-learning/
- https://docs.dask.org/en/latest/dataframe-best-practices.html
- 

In [ ]:
#!wget http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data/mortgage_2000-2016_1gb.tgz /home/jupyter/

In [1]:
%%bash
nvidia-smi
nvcc --version

Fri Dec 11 17:20:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    56W / 350W |      0MiB / 40537MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-SXM4-40GB      Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [2]:
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)
import cudf; print('cudf Version:', cudf.__version__)
import cuml; print('cudf Version:', cuml.__version__)
import gcsfs; print('gcsfs Version:', gcsfs.__version__)
import time
import glob
import dask; print('dask Version:', dask.__version__) 
import dask.dataframe as dask_df
import dask_cudf; print('dask cudf Version:', dask_cudf.__version__) 

numpy Version: 1.19.4
pandas Version: 1.1.5
XGBoost Version: 1.3.0-SNAPSHOT
cudf Version: 0.16.0
cudf Version: 0.16.0
gcsfs Version: 0.7.1
dask Version: 2020.12.0
dask cudf Version: 0.16.0


In [3]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
# Create a Dask Cluster with one worker per GPU
cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34923 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 179.38 GB


In [1]:
from dask.distributed import Client
client = Client()
client.scheduler_info()['services']
client

Client Scheduler: tcp://127.0.0.1:43585 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 24 Memory: 179.38 GB


Client Scheduler: tcp://127.0.0.1:43585 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 24 Memory: 179.38 GB


In [4]:
col_acq = ['LoanID','Channel','SellerName','OrInterestRate','OrUnpaidPrinc','OrLoanTerm',
        'OrDate','FirstPayment','OrLTV','OrCLTV','NumBorrow','DTIRat','CreditScore',
        'FTHomeBuyer','LoanPurpose','PropertyType','NumUnits','OccStatus','PropertyState',
        'Zip','MortInsPerc','ProductType','CoCreditScore','MortInsType','RelMortInd'];

col_per = ['LoanID','MonthRep','Servicer','CurrInterestRate','CAUPB','LoanAge','MonthsToMaturity',
          'AdMonthsToMaturity','MaturityDate','MSA','CLDS','ModFlag','ZeroBalCode','ZeroBalDate',
          'LastInstallDate','ForeclosureDate','DispositionDate','PPRC','AssetRecCost','MHRC',
          'ATFHP','NetSaleProceeds','CreditEnhProceeds','RPMWP','OFP','NIBUPB','PFUPB','RMWPF',
          'FPWA','ServicingIndicator'];

dtype={'LoanID': 'str',
'MonthRep': 'str',
'Servicer': 'str',
'CurrInterestRate': 'str',
'CAUPB': 'str',
'LoanAge': 'str',
'MonthsToMaturity': 'str',
'AdMonthsToMaturity': 'str',
'MaturityDate': 'str',
'MSA': 'str',
'CLDS': 'str',
'ModFlag': 'str',
'ZeroBalCode': 'str',
'ZeroBalDate': 'str',
'LastInstallDate': 'str',
'ForeclosureDate': 'str',
'DispositionDate': 'str',
'PPRC': 'str',
'AssetRecCost': 'str',
'MHRC': 'str',
'ATFHP': 'str',
'NetSaleProceeds': 'str',
'CreditEnhProceeds': 'str',
'RPMWP': 'str',
'OFP': 'str',
'NIBUPB': 'str',
'PFUPB': 'str',
'RMWPF': 'str',
'FPWA': 'str',
'ServicingIndicator': 'str'}



In [5]:
#df_acq = pd.read_csv('/home/jupyter/acq/Acquisition_2007Q4.txt', sep='|', names=col_acq, index_col=False)
#df_per = pd.read_csv('/home/jupyter/acq/perf/Performance_2007Q4.txt', sep='|', names=col_per, index_col=False)

In [6]:
acq_fnames='gs://mchrestkha-github-ml-examples/fannie_mae_loans/acq/*'
perf_fnames='gs://mchrestkha-github-ml-examples/fannie_mae_loans/perf/*'


In [5]:
!gsutil du -sh 'gs://mchrestkha-github-ml-examples/fannie_mae_loans/perf/'

/snap/google-cloud-sdk/161/lib/third_party/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
190.96 GiB   gs://mchrestkha-github-ml-examples/fannie_mae_loans/perf


In [6]:
%%time
#acq_ddf=dask_df.read_csv(fnames,sep='|',names=col_acq, assume_missing=True)
perf_ddf=dask_df.read_csv(perf_fnames,sep='|',names=col_per, assume_missing=True, dtype=dtype)
rows=perf_ddf.shape[0].compute()
cols=perf_ddf.shape[1]
print(rows,',', cols) 

1890353680 , 30
CPU times: user 2min 4s, sys: 11.5 s, total: 2min 15s
Wall time: 20min 48s


In [8]:
%%time
#acq_ddf=dask_df.read_csv(fnames,sep='|',names=col_acq, assume_missing=True)
perf_ddf=dask_cudf.read_csv(perf_fnames,sep='|',names=col_per, assume_missing=True, dtype=dtype)
rows=perf_ddf.shape[0].compute()
cols=perf_ddf.shape[1]
print(rows,',', cols) 

1890353680 , 30
CPU times: user 23.9 s, sys: 3.54 s, total: 27.5 s
Wall time: 14min 25s


In [ ]:
perf_df

In [ ]:
perf_df.shape

In [ ]:
perf_df.head()

In [ ]:
%%time
df = dask_cudf.read_csv(perf_fnames,sep='|',names=col_per, assume_missing=True, dtype=dtyp)
df.shape